In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR_old = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR_old.xml")
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Recon3D_Recon3D = read_sbml_model(Path().cwd() / "models" / "mod_Recon3D_Recon3D.xml")
mito = read_sbml_model(Path().cwd() /"models"/  "mitocore_12918_2017_500_MOESM4_ESM.xml")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [29]:
mod = read_sbml_model(Path('C:/Users/prins/GitHub/context_specific_models/basis_model/Human-GEM.xml'))

In [46]:
mod.compartments

{'c': 'Cytosol',
 'e': 'Extracellular',
 'l': 'Lysosome',
 'r': 'Endoplasmic reticulum',
 'm': 'Mitochondria',
 'x': 'Peroxisome',
 'g': 'Golgi apparatus',
 'n': 'Nucleus',
 'i': 'Inner mitochondria'}

In [44]:
mod.reactions.get_by_id('MAR10062')

Reaction identifier,MAR10062
Name,Protein pool for biomass reaction
Memory address,0x26dcfcc3970
Stoichiometry,0.0721 MAM02006c + 0.0801 MAM02335c + 0.0512 MAM02340c + 0.0375 MAM02341c + 0.0556 MAM02342c + 0.0183 MAM02351c + 0.0428 MAM02376c + 0.0783 MAM02377c + 0.0228 MAM02380c + 0.0442 MAM02401c + 0.0911... 0.0721 glycyl-tRNA(gly) + 0.0801 L-alanyl-tRNA(ala) + 0.0512 L-arginyl-tRNA(arg) + 0.0375 L-asparaginyl-tRNA(asn) + 0.0556 L-aspartyl-tRNA(asp) + 0.0183 L-cysteinyl-tRNA(cys) + 0.0428 L-glutaminyl-...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [39]:
[[m.id, m. ] for m in mod.reactions.get_by_id('MAR10062').products]

['MAM03063c',
 'MAM03064c',
 'MAM03065c',
 'MAM03066c',
 'MAM03067c',
 'MAM03068c',
 'MAM03069c',
 'MAM03070c',
 'MAM03071c',
 'MAM03072c',
 'MAM03073c',
 'MAM03074c',
 'MAM03075c',
 'MAM03076c',
 'MAM03077c',
 'MAM03078c',
 'MAM03079c',
 'MAM03080c',
 'MAM03081c',
 'MAM03082c',
 'MAM10013c']

In [35]:
from src.get_info import make_rxn_df, make_compact_rxn_df
df = make_rxn_df(mod)
df.to_clipboard()

In [31]:
mod.reactions.get_by_id('MAR10023')

Reaction identifier,MAR10023
Name,
Memory address,0x26dcfc8cc70
Stoichiometry,MAM03970c --> MAM03971e biomass --> biomass
GPR,
Lower bound,0.0
Upper bound,1000.0


In [32]:
mod.reactions.get_by_id('MAR10024')

Reaction identifier,MAR10024
Name,
Memory address,0x26dcfc8de40
Stoichiometry,MAM03971e <=> biomass <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [3]:
# modify models

# set objective function (PR ATP DM)
mod_RPE_PR_old.objective = 'MAR03964_PR'
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod_Recon3D_Recon3D.objective = 'DM_atp_c__PR'

# close PR exchange reactions, open RPE exchange upper boundaries
from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
models = [mod_RPE_PR, mod_Human1_Human1, mod_Recon3D_Recon3D, mod_RPE_PR_old]
for m in models:
    m = close_PR_EX(m)
    m = open_RPE_EX_ub(m)

In [4]:
for m in [mod_RPE_PR, mod_RPE_PR_old]:
    m = close_PR_EX(m)
    m = open_RPE_EX_ub(m)
    m.reactions.get_by_id('MAR09048_RPE').bounds=(-1000,1000)
    m.reactions.get_by_id('MAR09034_RPE').bounds=(-10,1000)
    m.reactions.get_by_id('MAR03964_RPE').bounds = (40,40)
    m.objective = 'MAR03964_PR'

In [5]:
mod_RPE_PR.optimize()

,fluxes,reduced_costs
MAR03905_RPE,0.0,0.000000
MAR03907_RPE,0.0,0.000000
MAR04281_RPE,0.0,0.000000
MAR04388_RPE,0.0,-0.000000
MAR04283_RPE,0.0,0.000000
...,...,...
MAR10065_PR,0.0,-21.184008
MAR10067_PR,0.0,0.000000
MAR10068_PR,0.0,0.000000
MAR13083_PR,0.0,0.000000


In [6]:
mod_RPE_PR_old.optimize()

,fluxes,reduced_costs
MAR03905_RPE,0.0,0.000000
MAR03907_RPE,0.0,0.000000
MAR04281_RPE,0.0,0.000000
MAR04388_RPE,0.0,-0.000000
MAR04283_RPE,0.0,0.000000
...,...,...
MAR10065_PR,0.0,-21.184008
MAR10067_PR,0.0,0.000000
MAR10068_PR,0.0,0.000000
MAR13083_PR,0.0,0.000000


In [11]:
mod_RPE_PR.reactions.get_by_id('MAR10023_RPE_PR')

Reaction identifier,MAR10023_RPE_PR
Name,
Memory address,0x26d34a2d9c0
Stoichiometry,MAM03970c_RPE --> MAM03971e_RPE_PR biomass --> biomass
GPR,
Lower bound,0.0
Upper bound,1000.0


In [232]:
from cobra.io import write_sbml_model
# save combined RPE_PR models
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")

In [210]:
mod_RPE_PR.objective = 'MAR03964_PR'

In [211]:
    mod_RPE_PR = close_PR_EX(mod_RPE_PR)
    mod_RPE_PR = open_RPE_EX_ub(mod_RPE_PR)

In [228]:
mod_RPE_PR.id = 'RPE_PR'

In [200]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Recon3D_dict = {'EX_o2s[e]_RPE':[(-10,1000)],'EX_glc_D[e]_RPE':[(-30,1000)],'DM_atp_c__RPE':[(0,0),(40,40),(80,80)]}
mitocore_dict = {'EX_o2_e':[(-1000,1000)],'EX_glc_D_e':[(-300,1000)]}

Human1_dict_p = create_permutation_dicts(Human1_dict)
Recon3D_dict_p = create_permutation_dicts(Recon3D_dict)
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

In [214]:
from src.modify_model import change_bounds
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE').bounds = (-10,1000)
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (-100,1000)

In [234]:
df = make_rxn_df(mod_RPE_PR)

In [236]:
df

,cell,lb,ub,vmhreaction,metanetx.reaction,reactome,ec-code,sbo,tcdb,rhea,bigg.reaction,kegg.reaction,name,subsystem,reaction,met_IDs,GPR
MAR03905_RPE,RPE,0.0,1000.0,ALCD2if,MNXR95725,,"[1.1.1.1, 1.1.1.71]",SBO:0000176,,"[25291, 25290]",ALCD2x,R00754,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NAD+[c_RPE] --> acetaldehyde[...,MAM01796c_RPE + MAM02552c_RPE --> MAM01249c_RP...,ENSG00000147576 or ENSG00000172955 or ENSG0000...
MAR03907_RPE,RPE,0.0,1000.0,ALCD2yf,MNXR95726,,1.1.1.2,SBO:0000176,,,ALCD2y,R00746,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,ethanol[c_RPE] + NADP+[c_RPE] --> acetaldehyde...,MAM01796c_RPE + MAM02554c_RPE --> MAM01249c_RP...,ENSG00000117448
MAR04281_RPE,RPE,-1000.0,1000.0,r0173,MNXR101040,,1.1.1.27,SBO:0000176,,,,R00703,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[x_RPE] + NADH[x_RPE] + pyruvate[x_RPE] <=> ...,MAM02039x_RPE + MAM02553x_RPE + MAM02819x_RPE ...,ENSG00000111716 or ENSG00000134333 or ENSG0000...
MAR04388_RPE,RPE,-1000.0,1000.0,LDH_L,MNXR101040,,1.1.1.27,SBO:0000176,,,LDH_L,R00703,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,H+[c_RPE] + NADH[c_RPE] + pyruvate[c_RPE] <=> ...,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,ENSG00000111716 or ENSG00000134333 or ENSG0000...
MAR04283_RPE,RPE,0.0,1000.0,ALDD2y,MNXR95750,,1.2.1.5,SBO:0000176,,"[25299, 25298]",,R00711,Acetaldehyde:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,acetaldehyde[c_RPE] + H2O[c_RPE] + NADP+[c_RPE...,MAM01249c_RPE + MAM02040c_RPE + MAM02554c_RPE ...,ENSG00000006534 or ENSG00000108602 or ENSG0000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR10065_PR,PR,0.0,1000.0,,,,,SBO:0000185,,,,,Cofactors and vitamins pool for biomass reaction,Artificial reactions,0.0345 11-cis-retinol[c_PR] + 0.0172 18-hydrox...,0.0345 MAM00291c_PR + 0.0172 MAM00427c_PR + 0....,
MAR10067_PR,PR,0.0,1000.0,,,,,SBO:0000176,,,,,,Inositol phosphate metabolism,"1D-myo-inositol-1,3,4,5,6-pentakisphosphate[c_...",MAM00521c_PR + MAM02040c_PR --> MAM00525c_PR +...,ENSG00000151151
MAR10068_PR,PR,0.0,1000.0,,,,,SBO:0000176,,,,,,Inositol phosphate metabolism,"1D-myo-inositol-1,4,5,6-tetrakisphosphate[c_PR...",MAM00525c_PR + MAM02040c_PR --> MAM01718c_PR +...,
MAR13083_PR,PR,0.0,1000.0,,,,,SBO:0000176,,,,,,Metabolism of other amino acids,H+[c_PR] + hypotaurine[c_PR] + NADH[c_PR] + O2...,MAM02039c_PR + MAM02157c_PR + MAM02553c_PR + M...,ENSG00000010932


In [235]:
df.to_clipboard()

In [201]:
def fba_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [169]:
def fba_fva_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            fva = flux_variability_analysis(m, loopless=False) #fva

            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            f_min[i] = fva['minimum']
            f_max[i] = fva['minimum']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [170]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000),(-40,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max, uptake, secretion = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

C:\Users\prins\AppData\Local\Temp\ipykernel_19124\3328211365.py:71: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_19124\3328211365.py:72: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_19124\3328211365.py:80: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


In [ ]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

In [ ]:
Human1_dict = create_permutation_dicts({'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0)]})
fba_fva_analysis(mod_RPE_PR, Human1_dict, 'MAR03964_PR')

In [ ]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048_RPE':[(-10,1000)],'MAR09034_RPE':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Human1_dict_p = create_permutation_dicts(Human1_dict)

fba_fva_analysis(mod_RPE_PR,Human1_dict_p, 'MAR03964_PR')